In [1]:
using Graphs
using Plots
using Base.Threads
using Random
include("../mpdbp.jl")
include("../inference/zero_patient.jl")
include("../bp_fast.jl")
import ProgressMeter; ProgressMeter.ijulia_behavior(:clear);

In [63]:
T = 20
N = 50
k = 4
gg = random_regular_graph(N, k)
g = IndexedGraph(gg)
t = SimpleGraph(prim_mst(gg))
g = IndexedGraph(t)
λ_true = 0.3
κ_true = 0.2
γ = 3/N
p⁰ = map(1:N) do i
    [1-γ, γ]
end;

In [64]:
sis_true = SIS(g, λ_true, κ_true, T; p⁰)
X, _ = onesample(mpdbp(sis_true));

In [65]:
lambdas = 0.01:0.01:0.5
kappas = 0.01:0.01:0.5
f = fill!(lambdas*kappas', 0);

In [66]:
# λ = λ_true
# κ = κ_true
# bp = mpdbp(SIS(g, λ, κ, T; p⁰))
# rng = MersenneTwister(0)
# x = draw_node_observations!(bp.ϕ, X, (T)*N; rng, softinf=1e2)
# iterate!(bp; maxiter=100, tol=1e-3, showprogress=false)

In [67]:
nobs = (T*N) ÷ 10

100

In [ ]:
for (i,λ) in enumerate(lambdas)
    println("## λ $i of ", length(lambdas))
    @threads for j in eachindex(kappas)
        κ = kappas[j]
        bp = mpdbp(SIS(g, λ, κ, T; p⁰))
        rng = MersenneTwister(0)
        draw_node_observations!(bp.ϕ, X, nobs; rng)
        iterate!(bp; maxiter=100, tol=1e-3, showprogress=false)
        f[i,j] = bethe_free_energy(bp)
    end
end

## λ 1 of 50
## λ 2 of 50
## λ 3 of 50
## λ 4 of 50
## λ 5 of 50
## λ 6 of 50
## λ 7 of 50
## λ 8 of 50
## λ 9 of 50
## λ 10 of 50
## λ 11 of 50
## λ 12 of 50
## λ 13 of 50
## λ 14 of 50
## λ 15 of 50
## λ 16 of 50
## λ 17 of 50
## λ 18 of 50
## λ 19 of 50
## λ 20 of 50
## λ 21 of 50
## λ 22 of 50
## λ 23 of 50
## λ 24 of 50
## λ 25 of 50
## λ 26 of 50
## λ 27 of 50
## λ 28 of 50
## λ 29 of 50
## λ 30 of 50


In [ ]:
heatmap(lambdas, kappas, exp.(-f)',title="likelihood of hyperparameters", xlabel="λ", ylabel="κ")
scatter!([λ_true], [κ_true], m=:plus, label="", c=:white, msw=4, ms=6)